# All Elastic Modulus Calculation Pathways

This notebook analyzes snow elastic modulus calculation methods at both **layer-level** and **slab-level** scales.

## Table of Contents

1. [Load Snow Pit Data](#1-load-snow-pit-data)
2. [Find All Elastic Modulus Calculation Pathways](#2-find-all-elastic-modulus-calculation-pathways)
3. [Layer-Level Analysis](#3-layer-level-analysis)
4. [Slab-Level Comparison (ECTP)](#4-slab-level-comparison-ectp)

**Target Parameter**: `elastic_modulus` — snow layer elastic modulus in Pa

Uncertainty reflects propagated input measurement uncertainties only (method regression standard error excluded): ±10% for direct density measurement, ±0.67 hand hardness index, ±0.5 mm grain size.

In [1]:
from pathlib import Path
from typing import Dict, Any
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from snowpyt_mechparams.snowpilot import parse_caaml_directory
from snowpyt_mechparams.data_structures import Pit, Slab
from snowpyt_mechparams.graph import graph
from snowpyt_mechparams.algorithm import find_parameterizations
from snowpyt_mechparams.execution import ExecutionEngine
from snowpyt_mechparams.execution.config import ExecutionConfig

## 1. Load Snow Pit Data

In [2]:
snow_pits_raw = parse_caaml_directory(str(Path("data")))
pits = [Pit.from_snow_pit(sp) for sp in snow_pits_raw]

print(f"Loaded {len(pits)} snow pits ({sum(len(pit.layers) for pit in pits)} layers)")

Loaded 50278 snow pits (371429 layers)


## 2. Find All Elastic Modulus Calculation Pathways

In [3]:
pathways = find_parameterizations(graph, graph.get_node("elastic_modulus"))

print(f"Found {len(pathways)} pathways for calculating elastic_modulus:\n")
for i, pathway in enumerate(pathways, 1):
    print(f"Pathway {i}:")
    print(pathway)
    print()

Found 16 pathways for calculating elastic_modulus:

Pathway 1:
branch 1: snow_pit -- data_flow --> measured_density -- data_flow --> density -- data_flow --> merge_density_grain_form
branch 2: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_density_grain_form
merge branch 1, branch 2: merge_density_grain_form -- bergfeld --> elastic_modulus

Pathway 2:
branch 1: snow_pit -- data_flow --> measured_hand_hardness -- data_flow --> merge_hand_hardness_grain_form
branch 2: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_hand_hardness_grain_form
branch 3: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_density_grain_form
merge branch 1, branch 2: merge_hand_hardness_grain_form -- geldsetzer --> density
merge branch 1, branch 2, branch 3: merge_density_grain_form -- bergfeld --> elastic_modulus

Pathway 3:
branch 1: snow_pit -- data_flow --> measured_hand_hardness -- data_flow --> merge_hand_hardness_grain_form
branch 2: snow_pit -

## 3. Layer-Level Analysis

Each layer is analyzed independently as a single-layer slab, regardless of its parent pit.

In [4]:
engine = ExecutionEngine(graph)
config = ExecutionConfig(include_method_uncertainty=False)

# Build flat list of (layer, slope_angle, pit_id, layer_index)
layer_infos = []
for pit in pits:
    try:
        angle = float(pit.slope_angle) if pit.slope_angle is not None and not np.isnan(pit.slope_angle) else 0.0
    except (TypeError, ValueError):
        angle = 0.0
    for idx, layer in enumerate(pit.layers):
        layer_infos.append((layer, angle, pit.pit_id, idx))

# Execute all pathways on each layer as a single-layer slab
all_results: Dict[str, Any] = {}
for layer, angle, pit_id, layer_idx in layer_infos:
    slab = Slab(layers=[layer], angle=angle, pit_id=pit_id)
    results = engine.execute_all(slab, "elastic_modulus", config=config)
    all_results[f"{pit_id}_L{layer_idx}"] = {
        'execution_results': results,
        'pit_id': pit_id,
    }

print(f"Executed {len(pathways)} pathways on {len(layer_infos)} layers")

Executed 16 pathways on 371429 layers


In [5]:
e_mod_data = []
for layer_id, info in all_results.items():
    for pathway_result in info['execution_results'].pathways.values():
        for trace in pathway_result.computation_trace:
            if trace.parameter == "elastic_modulus" and trace.success and trace.output is not None:
                out = trace.output
                if hasattr(out, 'nominal_value'):
                    val, std = out.nominal_value, out.std_dev
                else:
                    try:
                        val, std = float(out), 0.0
                    except (TypeError, ValueError):
                        continue
                density_method = pathway_result.methods_used.get('density', 'unknown')
                e_mod_method = pathway_result.methods_used.get('elastic_modulus', 'unknown')
                e_mod_data.append({
                    'layer_id': layer_id,
                    'pit_id': info['pit_id'],
                    'full_pathway': f"{density_method} \u2192 {e_mod_method}",
                    'elastic_modulus': val,
                    'elastic_modulus_std': std,
                })

df_e_mod = pd.DataFrame(e_mod_data)
df_e_mod['rel_unc'] = np.where(df_e_mod['elastic_modulus'] != 0, df_e_mod['elastic_modulus_std'] / df_e_mod['elastic_modulus'], np.nan)

# Summary table sorted by layer count descending
summary = (
    df_e_mod.groupby('full_pathway')
    .agg(layers=('layer_id', 'nunique'), avg_rel_unc=('rel_unc', 'mean'))
    .assign(coverage=lambda d: d['layers'] / df_e_mod['layer_id'].nunique())
    .sort_values('layers', ascending=False)
    .reset_index()
)

print(f"  {'Full Pathway':<50s} {'Layers':>8s} {'Coverage':>9s} {'Avg Rel. Uncertainty':>22s}")
print(f"  {'-'*91}")
for _, row in summary.iterrows():
    print(f"  {row['full_pathway']:<50s} {int(row['layers']):>8d} ({row['coverage']:>5.1%})    {row['avg_rel_unc']:>18.1%}")
print()
print("  Note: Uncertainty is propagated from input measurement uncertainties only.")

  Full Pathway                                         Layers  Coverage   Avg Rel. Uncertainty
  -------------------------------------------------------------------------------------------
  kim_jamieson_table2 → wautier                        205232 (88.5%)                 38.3%
  geldsetzer → schottner                               181780 (78.4%)                 76.7%
  kim_jamieson_table2 → schottner                      181780 (78.4%)                 83.3%
  geldsetzer → wautier                                 170727 (73.6%)                 39.7%
  kim_jamieson_table2 → kochle                         102219 (44.1%)                 66.6%
  kim_jamieson_table5 → wautier                         93515 (40.3%)                 42.1%
  kim_jamieson_table2 → bergfeld                        88658 (38.2%)                 77.4%
  geldsetzer → bergfeld                                 88032 (37.9%)                 74.1%
  kim_jamieson_table5 → schottner                       83986 (36.2%)      

## 4. Slab-Level Comparison (ECTP)

In [6]:
# Create ECTP slabs
ectp_slabs = []
for pit in pits:
    for slab in pit.create_slabs(weak_layer_def="ECTP_failure_layer"):
        ectp_slabs.append({'slab': slab, 'n_layers': len(slab.layers)})

print(f"Created {len(ectp_slabs)} ECTP slabs")

Created 14776 ECTP slabs


In [7]:
# Execute all elastic modulus pathways on each ECTP slab and count successes
# A slab succeeds for a pathway if ALL its layers have successful calculations
pathway_slab_success: Dict[str, int] = {}

for info in ectp_slabs:
    slab = info['slab']
    n = info['n_layers']
    results = engine.execute_all(slab, "elastic_modulus", config=config)
    for pathway_result in results.pathways.values():
        density_method = pathway_result.methods_used.get('density', 'unknown')
        e_mod_method = pathway_result.methods_used.get('elastic_modulus', 'unknown')
        full_pathway = f"{density_method} \u2192 {e_mod_method}"
        n_ok = sum(
            1 for t in pathway_result.computation_trace
            if t.parameter == "elastic_modulus" and t.success and t.output is not None
        )
        pathway_slab_success[full_pathway] = pathway_slab_success.get(full_pathway, 0) + (1 if n_ok == n else 0)

### Layer-Level vs Slab-Level Comparison

In [8]:
all_pathways = sorted(
    set(df_e_mod['full_pathway'].unique()) | set(pathway_slab_success.keys()),
    key=lambda p: df_e_mod[df_e_mod['full_pathway'] == p]['layer_id'].nunique() if p in df_e_mod['full_pathway'].values else 0,
    reverse=True,
)

total_layers = len(layer_infos)
total_slabs = len(ectp_slabs)

print(f"  {'Full Pathway':<50s} {'Layers':>22s} {'Slabs (ECTP)':>25s}")
print(f"  {'-'*99}")
for pathway in all_pathways:
    layer_n = df_e_mod[df_e_mod['full_pathway'] == pathway]['layer_id'].nunique() if pathway in df_e_mod['full_pathway'].values else 0
    layer_cov = layer_n / total_layers
    slab_n = pathway_slab_success.get(pathway, 0)
    slab_cov = slab_n / total_slabs
    print(f"  {pathway:<50s} {layer_n:>6d} ({layer_cov:>5.1%})    {slab_n:>6d} / {total_slabs} ({slab_cov:>5.1%})")

print()
print("  Slab success requires ALL layers in the slab to have successful calculations.")

  Full Pathway                                                       Layers              Slabs (ECTP)
  ---------------------------------------------------------------------------------------------------
  kim_jamieson_table2 → wautier                      205232 (55.3%)      2092 / 14776 (14.2%)
  geldsetzer → schottner                             181780 (48.9%)      2780 / 14776 (18.8%)
  kim_jamieson_table2 → schottner                    181780 (48.9%)      2780 / 14776 (18.8%)
  geldsetzer → wautier                               170727 (46.0%)      1607 / 14776 (10.9%)
  kim_jamieson_table2 → kochle                       102219 (27.5%)       525 / 14776 ( 3.6%)
  kim_jamieson_table5 → wautier                       93515 (25.2%)       552 / 14776 ( 3.7%)
  kim_jamieson_table2 → bergfeld                      88658 (23.9%)      1241 / 14776 ( 8.4%)
  geldsetzer → bergfeld                               88032 (23.7%)      1225 / 14776 ( 8.3%)
  kim_jamieson_table5 → schottner           